In [274]:
import time
import numpy as np
np.seterr(all='warn')
import scipy.special as sp
import math
from decimal import Decimal

In [275]:
# Valor de Pi de referência
PI = math.pi

# Iterações para o cálculo de Pi
# N_ITERATIONS = [10, 100, 1000, 10000, 100000, 1000000, 10000000, 100000000, 1000000000]
N_ITERATIONS = [10, 100, 1000, 10000, 100000, 1000000]


In [276]:
def factorial_numpy_large(n):
    """
    Calculates the factorial of a large number using NumPy's np.prod().
    This method leverages Python's arbitrary-precision integers for the result.
    """
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    if n == 0:
        return 1
    
    # Create an array of numbers from 1 to n (inclusive)
    # The 'object' dtype ensures Python's arbitrary-precision integers are used
    numbers = np.arange(1, n + 1, dtype=object) 
    
    # Calculate the product of all elements in the array
    return np.prod(numbers)

In [277]:
def emq(pi_approx):
    """Calcula o erro médio quadrático entre a aproximação de Pi e o valor de referência."""
    return (PI - pi_approx)**2

In [278]:
def leibniz_pi(n, dtype):
    """Calcula a aproximação de Pi usando a Série de Leibniz."""
    start = time.time()
    sum_series = dtype(0)
    for k in range(n):
        sum_series += dtype((-1)**k) / dtype(2*k + 1)
    pi = dtype(4) * sum_series
    end = time.time()
    return { "pi": pi, "time": end - start, "error": emq(pi) }

In [279]:
def nilakantha_pi(n, dtype):
    """Calcula a aproximação de Pi usando a Série de Nilakantha."""
    start = time.time()
    sum_series = dtype(0)
    for k in range(1, n+1):
        sum_series += dtype(4) / (dtype(2*k)*dtype(2*k+1)*dtype(2*k+2))
    pi = dtype(3) + sum_series
    end = time.time()
    return { "pi": pi, "time": end - start, "error": emq(pi) }

In [280]:
def machin_pi(n, dtype):
    """Calcula a aproximação de Pi usando a Fórmula de Machin."""
    start = time.time()
    pi = dtype(4)*dtype(np.tan(1/5))**(-1) - dtype(np.tan(1/239)**(-1))
    end = time.time()
    return { "pi": pi, "time": end - start, "error": emq(pi) }

In [281]:
def euler_pi(n, dtype):
    """Calcula a aproximação de Pi usando a Série de Euler."""
    start = time.time()
    sum_series = dtype(0)
    for k in range(1, n+1):
        sum_series += dtype(1) / dtype(k**2)
    pi = dtype(np.sqrt(6 * sum_series))
    end = time.time()
    return { "pi": pi, "time": end - start, "error": emq(pi) }

In [282]:
def ramanujan_pi(n, dtype):
    """Calcula a aproximação de Pi usando a Fórmula de Ramanujan."""
    start = time.time()
    pi = dtype(0)
    sum_series = dtype(0)
    for k in range(n):
        # Erro ao usar math.factorial: int too large to convert to float
        # Por isso, usamos uma função personalizada para calcular o fatorial
        # Erro: RuntimeWarning: overflow encountered in cast
        sum_series += (dtype(factorial_numpy_large(4*k)) * (dtype(1103) + dtype(26390*k))) / (dtype(factorial_numpy_large(k))**dtype(4) * dtype(396**(4*k)))
    factor = dtype(2) * dtype(np.sqrt(2)) / dtype(9801)
    pi = dtype(1) / dtype(factor * sum_series)
    end = time.time()
    return { "pi": pi, "time": end - start, "error": emq(pi) }

In [283]:
def chudnovsky_pi(n, dtype):
    """Calcula a aproximação de Pi usando a Fórmula de Chudnovsky."""
    start = time.time()
    sum_series = dtype(0)
    for k in range(n):
        # Erro: RuntimeWarning: overflow encountered in cast
        sum_series += (dtype((-1)**k) * dtype(factorial_numpy_large(6*k)) * (dtype(545140134*k) + dtype(13591409))) / (dtype(factorial_numpy_large(3*k) * dtype(factorial_numpy_large(k)**3) * dtype(640320**(3*k + 3/2))))
    result = dtype(12) * sum_series
    pi = dtype(1) / result
    end = time.time()
    return { "pi": pi, "time": end - start, "error": emq(pi) }

In [284]:
def bbp_pi(n, dtype):
    """Calcula a aproximação de Pi usando a fórmula de Bailey-Borwein-Plouffe."""
    start = time.time()
    sum_series = dtype(0)
    for k in range(n):
        sum_series += (dtype(1) / (dtype(16)**k))*(dtype(4) / (dtype(8*k + 1))-dtype(2) / (dtype(8*k + 4))-dtype(1) / (dtype(8*k + 5))-dtype(1) / (dtype(8*k + 6)))
    pi = sum_series
    end = time.time()
    return { "pi": pi, "time": end - start, "error": emq(pi) }

In [285]:
methods = {
    "Leibniz": leibniz_pi,
    "Nilakantha": nilakantha_pi,
    "Machin": machin_pi,
    "Euler": euler_pi,
    "Ramanujan": ramanujan_pi,
    "Chudnovsky": chudnovsky_pi,
    "BBP": bbp_pi
}

dtypes = {
    "float": np.float32,
    "double": np.float64
}

In [286]:
print("\n==== Resultados (Erro Médio Quadrático - EMQ) ====\n")


==== Resultados (Erro Médio Quadrático - EMQ) ====



In [287]:
for dtype_name, dtype in dtypes.items():
    print(f"\nTipo de variável: {dtype_name}\n")
    print(f"{'Método':<15}{'N':>10}{'Pi Aproximado':>20}{'EMQ':>20}{'Tempo (s)':>20}")
    print("-" * 100)
    for method_name, method_func in methods.items():
        for N in N_ITERATIONS:
            try:
                result = method_func(N, dtype=dtype)
                pi_a = result["pi"]
                error = result["error"]
                elapsed_time = result["time"]
                print(f"{method_name:<15}{N:>10}{pi_a:>20.15f}{error:>20.10e}{elapsed_time:>20.10f}")
            except Exception as e:
                print(f"{method_name:<15}{N:>10}{'Erro':>20}{'Erro':>20}{'Erro':>20}")
                print(e)
                print("-" * 100)
    print("-" * 100)



Tipo de variável: float

Método                  N       Pi Aproximado                 EMQ           Tempo (s)
----------------------------------------------------------------------------------------------------
Leibniz                10   3.041839599609375    9.9506890401e-03        0.0000193119
Leibniz               100   3.131592512130737    1.0000457405e-04        0.0000457764
Leibniz              1000   3.140592575073242    1.0003318494e-06        0.0004694462
Leibniz             10000   3.141498565673828    8.8689944278e-09        0.0057570934
Leibniz            100000   3.141585826873779    4.7805315262e-11        0.0526015759
Leibniz           1000000   3.141595363616943    6.8780536822e-12        0.5445201397
Nilakantha             10   3.225347280502319    7.0148226805e-03        0.0000171661
Nilakantha            100   3.227386713027954    7.3606055230e-03        0.0000829697
Nilakantha           1000   3.227410078048706    7.3646153323e-03        0.0008361340
Nilakantha   

C:\Users\mello\AppData\Local\Temp\ipykernel_3520\3976834385.py:10: RuntimeWarning: overflow encountered in cast
  sum_series += (dtype(factorial_numpy_large(4*k)) * (dtype(1103) + dtype(26390*k))) / (dtype(factorial_numpy_large(k))**dtype(4) * dtype(396**(4*k)))
C:\Users\mello\AppData\Local\Temp\ipykernel_3520\3976834385.py:10: RuntimeWarning: overflow encountered in scalar multiply
  sum_series += (dtype(factorial_numpy_large(4*k)) * (dtype(1103) + dtype(26390*k))) / (dtype(factorial_numpy_large(k))**dtype(4) * dtype(396**(4*k)))
C:\Users\mello\AppData\Local\Temp\ipykernel_3520\3976834385.py:10: RuntimeWarning: invalid value encountered in scalar divide
  sum_series += (dtype(factorial_numpy_large(4*k)) * (dtype(1103) + dtype(26390*k))) / (dtype(factorial_numpy_large(k))**dtype(4) * dtype(396**(4*k)))
C:\Users\mello\AppData\Local\Temp\ipykernel_3520\3976834385.py:10: RuntimeWarning: overflow encountered in scalar power
  sum_series += (dtype(factorial_numpy_large(4*k)) * (dtype(1103) 

BBP                100000   3.141592502593994    5.6843418861e-14        0.2749607563
BBP               1000000   3.141592502593994    5.6843418861e-14        2.6405534744
----------------------------------------------------------------------------------------------------

Tipo de variável: double

Método                  N       Pi Aproximado                 EMQ           Tempo (s)
----------------------------------------------------------------------------------------------------
Leibniz                10   3.041839618929403    9.9506679240e-03        0.0000052452
Leibniz               100   3.131592903558554    9.9995000687e-05        0.0000166893
Leibniz              1000   3.140592653839794    9.9999950000e-07        0.0001950264
Leibniz             10000   3.141492653590034    9.9999999517e-09        0.0025091171
Leibniz            100000   3.141582653589720    1.0000000147e-10        0.0237479210
Leibniz           1000000   3.141591653589774    1.0000000376e-12        0.26007986